In [13]:
import requests
from tqdm.notebook import tqdm
import json

In [14]:
base_url = "http://0.0.0.0:5000"

api_endpoints = [
    base_url + "/api/inspect/describe_openaire?url=",
    base_url + "/api/inspect/describe_opencitation?url=",
    base_url + "/api/inspect/describe_wikidata?url=",
]

In [15]:
NB_DOIS = 100

%run get_dois_dryad.ipynb
filename = "dryad_" + str(NB_DOIS) + ".json"

# %run get_dois_pangaea.ipynb
# filename = "pangaea_" + str(NB_DOIS) + ".json"

# %run get_dois_zenodo.ipynb
# filename = "zenodo_" + str(NB_DOIS) + ".json"

%store -r dois_list

REST request to dryad...
Parsing result...
https://datadryad.org/api/v2/datasets?page=2&per_page=100
REST request to dryad...
https://datadryad.org/api/v2/datasets?page=3&per_page=100
REST request to dryad...
https://datadryad.org/api/v2/datasets?page=4&per_page=100
REST request to dryad...
https://datadryad.org/api/v2/datasets?page=5&per_page=100
REST request to dryad...
https://datadryad.org/api/v2/datasets?page=6&per_page=100
REST request to dryad...
https://datadryad.org/api/v2/datasets?page=7&per_page=100
REST request to dryad...
https://datadryad.org/api/v2/datasets?page=8&per_page=100
REST request to dryad...
https://datadryad.org/api/v2/datasets?page=9&per_page=100
REST request to dryad...
https://datadryad.org/api/v2/datasets?page=10&per_page=100
REST request to dryad...
https://datadryad.org/api/v2/datasets?page=11&per_page=100
REST request to dryad...
https://datadryad.org/api/v2/datasets?page=12&per_page=100
REST request to dryad...
https://datadryad.org/api/v2/datasets?pag

In [16]:
import random
#sample_tool = random.choice(index.keys())

random.seed(10)

print(len(dois_list))

#samples = random.sample(list(index.items()), 1000)
samples_dois_list = random.sample(dois_list, NB_DOIS)

splitedSize = 100
samples_chunks = [samples_dois_list[x:x+splitedSize] for x in range(0, len(samples_dois_list), splitedSize)]

3100


In [17]:
print(len(samples_dois_list))

100


In [24]:
results = []
for url in tqdm(samples_dois_list):
    response = requests.get(base_url + "/api/inspect/get_rdf_metadata?url=" + url)
    graph = json.dumps(response.json(), ensure_ascii=False)
    for endpoint in api_endpoints:
        response = requests.post(endpoint.replace("?url=", "/"), json={"json-ld": graph, "url": url})
        result = {
            "url": url,
            "endpoint": endpoint.split("/")[-2],
            "triples_before": response.json()["triples_before"],
            "triples_after": response.json()["triples_after"],
            "improved": response.json()["triples_before"] < response.json()["triples_after"],
        }
        results.append(result)

  0%|          | 0/100 [00:00<?, ?it/s]

In [ ]:
improved = 0
not_improved = 0

for res in results:
    if not res["improved"]:
        not_improved += 1
    else:
        print("ENDPOINT: " + res["endpoint"])
        print("URL: " + res["url"])
        improved += 1
        
print("Improved: " + str(improved))
print("Not improved: " + str(not_improved))

ENDPOINT: describe_openaire
URL: https://doi.org/10.1594/PANGAEA.785298
ENDPOINT: describe_openaire
URL: https://doi.org/10.1594/PANGAEA.874463
ENDPOINT: describe_openaire
URL: https://doi.org/10.1594/PANGAEA.821717
ENDPOINT: describe_openaire
URL: https://doi.org/10.1594/PANGAEA.770070
ENDPOINT: describe_openaire
URL: https://doi.org/10.1594/PANGAEA.861691
ENDPOINT: describe_openaire
URL: https://doi.org/10.1594/PANGAEA.751391
ENDPOINT: describe_openaire
URL: https://doi.org/10.1594/PANGAEA.839181
ENDPOINT: describe_openaire
URL: https://doi.org/10.1594/PANGAEA.510801
ENDPOINT: describe_openaire
URL: https://doi.org/10.1594/PANGAEA.892787
ENDPOINT: describe_openaire
URL: https://doi.org/10.1594/PANGAEA.823111
ENDPOINT: describe_openaire
URL: https://doi.org/10.1594/PANGAEA.745448
ENDPOINT: describe_openaire
URL: https://doi.pangaea.de/10.1594/PANGAEA.806685
ENDPOINT: describe_openaire
URL: https://doi.pangaea.de/10.1594/PANGAEA.810727
ENDPOINT: describe_openaire
URL: https://doi.org/1

In [ ]:
json_object = json.dumps(results)
with open("results/" + filename, "w+") as outfile:
    outfile.write(json_object)